# Transformata Fouriera dla obrazów cyfrowych. Filtracja w dziedzinie częstotliwości.

## Cel ćwiczenia

- Zapoznanie z wykorzystaniem transformaty Fouriera w przetwarzaniu obrazów cyfrowych.
- Zapoznanie z pojęciem F-obrazu (amplitudy i fazy).
- Zapoznanie z własnościami transformaty Fouriera.
- Zapoznanie z filtracją w dziedzinie częstotliwości.

Na jednym z poprzednich ćwiczeń zetknęliśmy się z pojęciem konwolucji.
Przykładem może być filtracja dolno i górnoprzepustowa.
Operacja ta odpowiada mnożeniu w dziedzinie częstotliwości zgodnie z zależnością:

\begin{equation}
\mathcal{F}(g(x,y)*h(x,y)) = \mathcal{F}(g(x,y)) \cdot \mathcal{F}(h(x,y))
\end{equation}

gdzie: $\mathcal{F}$ oznacza transformatę Fouriera, a $*$ jest splotem.

Operacja filtracji w dziedzinie częstotliwości może okazać się bardziej efektywna, jeżeli operacje $fft$ i $ifft$ (odpowiednio szybka transformata Fouriera -- *fast Fourier transform* -- oraz odwrotna szybka transformata Fouriera -- *inverse fast Fourier transform*) zajmą mniej czasu niż klasyczna konwolucja (zazwyczaj ma to miejsce dla dużego obrazu, z dużą maską).

Sama filtracja w dziedzinie częstotliwości to mnożenie punktowe całego obrazu przez jedną maskę.

W przypadku filtracji w dziedzinie częstotliwości zakłada się, że obraz "zawija się" na brzegach, co może powodować pewne artefakty (zostanie to pokazane w trakcie ćwiczenia).

W dziedzinie częstotliwości "działają" tylko filtry liniowe.
Filtry medianowe, maksymalne, minimalne itp. nie mają swoich odpowiedników.

## Dwuwymiarowa transformata Fouriera

1. Wczytaj plik "dwieFale.bmp" w skali szarości.
Jest to obraz powstały na podstawie następującej zależności:

\begin{equation}
L(m, n) = 128 + 127 \cdot \cos(\frac{2\pi m}{32}+\frac{3\pi}{4}) \cdot \cos(\frac{2\pi n}{8}-\frac{\pi}{2})
\end{equation}<br>

gdzie: $m$ i $n$ są odpowiednio numerami wierszy i kolumn.

2. Do realizacji dwuwymiarowej transformaty Fouriera służy funkcja `cv2.dft`.
Ustaw flagę `flags=cv2.DFT_COMPLEX_OUTPUT`.
Wykonaj transformatę na wczytanym obrazie.
W ten sposób uzyskuje się tzw. F-obraz.

3. Najniższe częstotliwości znajdują się w lewym-górnym rogu obrazu.
Dla celów wizualizacji (ale też przetwarzania) często wykonuje się tzw. przesunięcie F-obrazu, które powoduje, że niskie częstotliwości przesuwane są do środka obrazu.
Wykorzystaj funkcję `np.fft.fftshift`.
Jako pierwszy argument podaj wynik transformaty Fouriera.
Jako drugi argument podaj numery osi, wzdłuż których należy wykonać operację.
Pierwsza oś odnosi się do wierszy obrazu.
Druga oś odnosi się do kolumn obrazu.
Trzecia oś to część rzeczywista (`[:, :, 0]`) lub urojona (`[:, :, 1]`).
W naszym przypadku argument powinien wyglądać tak `[0,1]`.

4. Wyświetl wynik transformaty.
Na wspólnym wykresie umieść obraz oryginalny, amplitudę i fazę F-obrazu.
Amplitudę i fazę wyznacz za pomocą funkcji `cv2.cartToPolar`.
Pierwszym argumentem funkcji jest część rzeczywista wyniku, a drugim urojona.
Uwaga. W razie wątpliwości proszę sprawdzić rozmiary rezultatu transformaty Fouriera oraz przesunięcia.

5. Dla wizualizacji oblicz logarytm dziesiętny amplitudy: `ALog = np.log10(A + 1)`.
Wyświetl go zamiast amplitudy na poprzednim wykresie.

6. Wczytaj obrazy *kolo.bmp*, *kwadrat.bmp*, *kwadrat45.bmp*, *trojkat.bmp*.
Czy analizując F-obraz można coś powiedzieć o kierunku krawędzi obiektów?

7. Sprawdź (empirycznie) poprawność stwierdzenia:
>Dwuwymiarowa transformata Fouriera jest złożeniem dwóch transformat jednowymiarowych (wykonanych np. najpierw wierszowo, a później kolumnowo). Jednowymiarowa transformata realizowana jest za pomocą funkcji fft (z bibloteki Numpy).
>
Wykonaj najpierw transformatę po wierszach: `FRow = np.fft.fft(I, axis=0)`.
Następnie po kolumnach: `FCol = np.fft.fft(I, axis=1)`.
Porównaj tak uzyskany wynik z rezultatem działania funkcji `cv2.dft`.
Można to zrobić wizualnie lub z wykorzystaniem funkcji `cv2.absdiff`.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("dwieFale.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/dwieFale.bmp
if not os.path.exists("kolo.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kolo.bmp
if not os.path.exists("kwadrat.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat.bmp
if not os.path.exists("kwadrat45.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat45.bmp
if not os.path.exists("kwadratKL.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratKL.bmp
if not os.path.exists("kwadratS.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratS.bmp
if not os.path.exists("kwadratT.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratT.bmp
if not os.path.exists("lena.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp
if not os.path.exists("trojkat.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/trojkat.bmp
        
I_Fale = cv2.imread('dwieFale.bmp', cv2.IMREAD_GRAYSCALE)


F_fale = cv2.dft(np.float32(I_Fale), flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(F_fale, [0,1])
mag, ang = cv2.cartToPolar(dft_shift[..., 0], dft_shift[..., 1])
Alog = np.log10(mag + 1)


figFale, axsFale = plt.subplots(1, 3, figsize=(15,15))
axsFale[0].imshow(I_Fale, cmap = 'gray')
axsFale[0].axis('off')
axsFale[1].imshow(mag, cmap = 'gray')
axsFale[1].axis('off')
axsFale[2].imshow(ang, cmap = 'gray')
axsFale[2].axis('off')
figFale.show()
figFale, axsFale = plt.subplots(1, 3, figsize=(15,15))
axsFale[0].imshow(I_Fale, cmap = 'gray')
axsFale[0].axis('off')
axsFale[1].imshow(Alog, cmap = 'gray')
axsFale[1].axis('off')
axsFale[2].imshow(ang, cmap = 'gray')
axsFale[2].axis('off')
figFale.show()

In [ ]:
trojkat = cv2.imread('trojkat.bmp', cv2.IMREAD_GRAYSCALE)
FRow = np.fft.fft(trojkat, axis=0)
fft =  np.fft.fft(FRow, axis=1)
dft_shift = np.fft.fftshift(fft, [0,1])
mag, ang = cv2.cartToPolar(dft_shift[..., 0], dft_shift[..., 1])
Alog = np.log10(mag + 1)
figFale, axsFale = plt.subplots(1, 3, figsize=(15,15))
axsFale[0].imshow(image, cmap = 'gray')
axsFale[0].axis('off')
axsFale[1].imshow(Alog, cmap = 'gray')
axsFale[1].axis('off')
axsFale[2].imshow(ang, cmap = 'gray')
axsFale[2].axis('off')
figFale.show()

In [ ]:
kolo = cv2.imread('kolo.bmp', cv2.IMREAD_GRAYSCALE)
kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)
trojkat = cv2.imread('trojkat.bmp', cv2.IMREAD_GRAYSCALE)

def dft(image):
    dft = cv2.dft(np.float32(image), flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft, [0,1])
    mag, ang = cv2.cartToPolar(dft_shift[..., 0], dft_shift[..., 1])
    Alog = np.log10(mag + 1)
    figFale, axsFale = plt.subplots(1, 3, figsize=(15,15))
    axsFale[0].imshow(image, cmap = 'gray')
    axsFale[0].axis('off')
    axsFale[1].imshow(Alog, cmap = 'gray')
    axsFale[1].axis('off')
    axsFale[2].imshow(ang, cmap = 'gray')
    axsFale[2].axis('off')
    figFale.show()
dft(kolo)
dft(kwadrat)
dft(kwadrat45)
dft(trojkat)

## Własności dwuwymiarowej transformaty Fouriera

1. Zbadaj jak zmienia się F-obraz (amplituda i faza) podczas następujących operacji: translacja, rotacja, zmiana rozmiaru, kombinacja liniowa.
Wykorzystaj stworzony wcześniej kod.
Uwaga. Należy użyć przygotowanych obrazów, a nie "generować" własne.
2. Do badania translacji wykorzystaj obrazy *kwadrat.bmp* i *kwadratT.bmp*.
3. Przy badaniu rotacji wykorzystaj obrazy *kwadrat.bmp* i *kwadrat45.bmp*.
4. Przy badaniu zmiany rozmiaru wykorzystaj obrazy *kwadrat.bmp* i *kwadratS.bmp*.
5. Przy badaniu kombinacji liniowej wykorzystaj obrazy *kwadrat.bmp*, *kwadrat45.bmp* i *kwadratKL.bmp*.

In [ ]:
import imutils

kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)
kwadratT = cv2.imread('kwadratT.bmp', cv2.IMREAD_GRAYSCALE)
kwadratS = cv2.imread('kwadratS.bmp', cv2.IMREAD_GRAYSCALE)
kwadratKL = cv2.imread('kwadratKL.bmp', cv2.IMREAD_GRAYSCALE)


def translation(image):
    h, w = image.shape[:2] 
    qh, qw = h / 4, w / 4
    T = np.float32([[1, 0, qw], [0, 1, qh]]) 
    translated = cv2.warpAffine(image, T, (w, h))
    return translated

def rotation(image, a = 45):
    rotated = imutils.rotate(image, angle= a)
    return rotated

def resize(image, scale = 50):
    w_scale = int(image.shape[1] * scale / 100)
    h_scale = int(image.shape[0] * scale / 100)
    dsize = (w_scale, h_scale)
    resized = cv2.resize(image, dsize)
    
    return resized

def linear_comb(image1, image2, image3, alpha = 1, beta = 1, theta = 1):
    result = alpha * image1 + beta * image2 + theta * image3
    result = np.around(result).astype(np.uint8)
    return result

dft(translation(kwadrat))
dft(translation(kwadratT))

dft(rotation(kwadrat))
dft(rotation(kwadrat45))

dft(resize(kwadrat))
dft(resize(kwadratS))

dft(linear_comb(kwadrat, kwadrat45, kwadratKL))

## Odwrotna dwuwymiarowa transformata Fouriera

1. Wykorzystaj stworzony wcześniej kod. Wybierz dowolny obraz np "kolo.bmp".
2. Przed realizacją odwrotnego przekszałcenia należy wykonać odwrotne przesunięcie.
Wykorzystaj funkcję `np.fft.ifftshift`.
Pierwszym argumentem jest wynik transformaty Fouriera.
Drugim argumentem są numery osi, wzdłuż których należy wykonać operację.
3. Wykonaj odwrotną transformatę Fouriera za pomocą funkcji `cv2.idft`.
4. Wyświetl wynik.
Sprawdź (wizualnie i poprzez odjęcie) czy obraz oryginalny i po przekształceniach są takie same.

In [ ]:
def idft(image):
    dft = cv2.dft(np.float32(image), flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft, [0,1])
    idft_shift = np.fft.ifftshift(dft_shift, [0,1])
    idft = cv2.idft(idft_shift)
    idft = cv2.magnitude(idft[:,:,0],idft[:,:,1])
    idft = cv2.normalize(idft, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    fig, axs = plt.subplots(1, 3, figsize=(15,15))
    axs[0].imshow(image, cmap = 'gray')
    axs[0].axis('off')
    axs[0].set_title('orginal')
    axs[1].imshow(idft, cmap = 'gray')
    axs[1].axis('off')
    axs[1].set_title('idft')
    axs[2].imshow(np.subtract(image, idft), cmap = 'gray')
    axs[2].axis('off')
    axs[2].set_title('substract')
    fig.show()
    
idft(kolo)

## Filtracja obrazu w dziedzinie częstotliwości

1. Wczytaj obraz "lena.bmp" w skali szarości.
Wykonaj transformatę Fouriera.
Wykorzystaj stworzony poprzednio kod.
Wyświetl obraz oryginalny, amplitudę (w skali logarytmicznej) i fazę.

2. Przeprowadź filtrację dolnoprzepustową - usuń górne częstotliwości.
Dla F-obrazu po operacji przesunięcia (`fftshift`) niskie częstotliwości leżą w jego centrum.

3. Na początku stwórz filtr "kołowy", dolnoprzepustowy.
Należy wygenerować macierze opisujące przestrzeń w dziedzinie częstotliwości.
Ich rozmiar musi być taki sam jak rozmiar przetwarzanego obrazu.
        lenaSize = I_Lena.shape
        FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
        FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
        FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
        FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)
Powyższy kod wygeneruje dwie znormalizowane macierze częstotliwości: *FSpaceRowsM* i *FSpaceColsM*.
Następnie należy wyznaczyć macierz zawierającą "odległość" od składowej stałej.
        FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

Uwagi:
- funkcja `fftfreq` generuje wektor częstotliwości $[-0.5, 0.5]$ o określonym rozmiarze, przy czym układ wartości jest taki, że najpierw od 0 do 0.5, a potem od -0.5 do 0,
- operacja `fftshift` zmienia ten układ na $[-0.5, 0.5]$,
- mnożenie przez 2 ustala ostatecznie zakres na $[-1, 1]$,
- operacja `outer` to tzw. iloczyn zewnętrzy dwóch wektorów, w naszym przypadku powoduje, że wektor pionowy lub poziomy jest "powielany" odpowiednią liczbę razy.   
- sugeruje się, aby przyglądnąć się jak wygląda macierz `FreqR` - czy to w debugerze, czy poprzez wizualizację.

4. Teraz należy wybrać interesujący zakres.
Tu można zdefiniować typ filtru (dolno, górno, pasmowoprzepustowy).

        FilterF = FreqR <= 0.1

Filtr należy zwizualizować:

        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()

4. Wykonaj właściwą filtrację, czyli mnożenie F-obrazu przez filtr FilterF.
Trzeba pamiętać, że F-obraz ma 2 kanały (rzeczywisty i urojony).
By mnożenie było możliwe należy więc powielić filtr również na 2 kanały.

        FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

5. Wykonaj operację odwrotnego przesunięcia i odwrotnej transformaty.
Oblicz wartość bezwzględną wyniku.
Wykorzystaj funkcję `cv2.magnitude`.
Pierwszym argumentem jest część rzeczywista.
Drugim argumentem jest część urojona.
Wynik wyświetl.

6. Poeksperymentuj z rozmiarem filtru (promieniem).
Zaimplementuj filtr górnoprzepustowy (zmiana znaku przy warunku na odległość) oraz pasmowoprzepustowy (dwa warunki na promień połączone operatorem AND '&' ).
Wykonaj co najmniej trzy filtry i wyświetl wyniki.

7. W ten sposób zaimplementowana filtracja wprowadza pewne artefakty w postaci "pierścieni" wokół krawędzi.
Zapobiec temu zjawisku można poprzez odpowiednie "modelowanie" filtra.
W tym celu wykorzystać należy okna, np. Hamminga, Hanninga, Chebysheva (znane z przetwarzania sygnałów 1D).
Zagadnienie to jest tematem zadania domowego do tego ćwiczenia.

In [ ]:
lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
plt.gray()

dft(lena)
lenaSize = lena.shape
FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
FilterF = FreqR <= 0.1

figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
figFilter.show()



In [ ]:
lp_filter= np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

lena_dft = cv2.dft(np.float32(lena), flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(lena_dft)
filterd = dft_shift * lp_filter
idft_shift = np.fft.ifftshift(filterd)
idft_lena1 = cv2.idft(idft_shift)
idft_lena1 = cv2.magnitude(abs(idft_lena1[:,:,0]), abs(idft_lena1[:,:,1]))


hp_filter_ = FreqR >= 0.2
hp_filter = np.repeat(hp_filter_[:, :, np.newaxis], 2, axis=2)
filterd = dft_shift * hp_filter
idft_shift = np.fft.ifftshift(filterd)
idft_lena2 = cv2.idft(idft_shift)
idft_lena2 = cv2.magnitude(abs(idft_lena2[:,:,0]), abs(idft_lena2[:,:,1]))


p_filer_ = ((FreqR >= 0.2) & (FreqR <= 0.4))
p_filer = np.repeat(p_filer_[:, :, np.newaxis], 2, axis=2)
filterd = dft_shift * p_filer
idft_shift = np.fft.ifftshift(filterd)
idft_lena3 = cv2.idft(idft_shift)
idft_lena3 = cv2.magnitude(abs(idft_lena3[:,:,0]), abs(idft_lena3[:,:,1]))


fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(idft_lena1, cmap = 'gray')
axs[0].axis('off')
axs[0].set_title('dolnoprzepustowy')
axs[1].imshow(idft_lena2, cmap = 'gray')
axs[1].axis('off')
axs[1].set_title('gornoprzepustowy')
axs[2].imshow(idft_lena2, cmap = 'gray')
axs[2].axis('off')
axs[2].set_title('pasmoprzepustowy')
fig.show()

## Implementacja wyszukiwania wzorca za pomocą FFT

1. Wczytaj w skali szarości i wyświetl obrazy *literki.bmp* i *wzorA.bmp*.

2. Wyznacz transformatę Fouriera obrazu *literki.bmp*.

3. Obróć drugi obraz o $180^\circ$.
Zastosuj funkcję `np.rot90`.
Pierwszym argumentem jest obracana macierz, a drugim liczba obrotów o $90^\circ$.

4. Należy wyznaczyć transformatę Fouriera obróconego obrazu w taki sposób, żeby miała ona taki sam rozmiar jak pierwszy obraz.
W tym celu należy zastosować *Zero Padding*.
Operacja ta polega na uzupełnieniu obrazu zerami do oczekiwanego rozmiaru.
Uzupełnij obraz zerami z **prawej** strony i z **dołu**.
W tym celu należy wykorzystać funkcję `cv2.copyMakeBorder`.
    - Pierwszym argumentem jest obraz wejściowy.
    - Drugim argumentem jest liczba wierszy u góry.
    - Trzecim argumentem jest liczba wierszy u dołu.
    - Czwartym argumentem jest liczba kolumn z lewej.
    - Piątym argumentem jest liczba kolumn z prawej.
    - Szóstym argumentem jest flaga typu wypełnienia.
    Dla stałej wartości podaj `cv2.BORDER_CONSTANT`.
    - Siódmym argementem jest wartość pikseli w ramce.
    Przekaż `value=0`.

5. Wyznacz transformatę Fouriera obrazu stworzonego w poprzednim punkcie.

6. Wyniki obu transformat należy przekonwertować do liczb zespolonych.
Obecnie jest to dwukanałowa macierz.
Pierwszy kanał odpowiada za część rzeczywistą.
Drugi kanał odpowiada za część urojoną.
Aby to osiągnąć wystarczy wykonać działanie:
        Complex = Real + Imag * 1j

7. Przemnóż ze sobą zespolone wyniki transformat.

8. Wynik należy powrotnie przekształcić do dwukanałowej macierzy.
Aby to zrobić wykonaj operację:
        CompMat = cv2.merge([np.real(Complex), np.imag(Complex)])

9. Wykonaj odwrotną transformatę Fouriera.
Dodaj flagę `flags=cv2.DFT_COMPLEX_INPUT`.

10. Oblicz wartość bezwzględną wyniku.

11. Wykonaj morfologiczną operację **Top-Hat**, by znaleźć maksima lokalne.
Operacja ta zostanie dokładnej wyjaśniona w jednym z kolejnych ćwiczeń.
W tym celu wykorzystaj operację:
        cv2.morphologyEx(correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

12. Wyświetl obok siebie obraz wejściowy i wynik wykonanych operacji.
Czy możesz wskazać położenie wzoru na podstawie drugiego obrazu?

In [ ]:
if not os.path.exists("literki.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/literki.bmp

if not os.path.exists("wzorA.bmp") :
    !wget--no-check-certificate https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/wzorA.bmp
wzorA = cv2.imread('wzorA.bmp', cv2.IMREAD_GRAYSCALE)
literki = cv2.imread('literki.bmp', cv2.IMREAD_GRAYSCALE)

literki_dft = cv2.dft(np.float32(literki), flags = cv2.DFT_COMPLEX_OUTPUT)
wzorA_rot = np.rot90(wzorA,2)

l_width, l_height = literki.shape
w_width, w_height = wzorA.shape

cpy_wzor = cv2.copyMakeBorder(wzorA_rot, 0, l_width - w_width, 0, l_height - w_height, cv2.BORDER_CONSTANT, value=0)
wzorA_dft = cv2.dft(np.float32(cpy_wzor), flags = cv2.DFT_COMPLEX_OUTPUT)

cmplx_wzor = np.fft.fftshift(wzorA_dft[:, :, 0]) + np.fft.fftshift(wzorA_dft[:, :, 1]) * 1j
cmplx_lit = np.fft.fftshift(literki_dft[:, :, 0]) + np.fft.fftshift(literki_dft[:, :, 1]) * 1j

cmplx_mul = cmplx_wzor*cmplx_lit
CompMat = cv2.merge([np.real(cmplx_mul), np.imag(cmplx_mul)])
idft = abs(cv2.idft(CompMat, flags=cv2.DFT_COMPLEX_INPUT))
idft = cv2.magnitude(idft[:,:,0], idft[:,:,1])
cv2.morphologyEx(idft, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))
fig, axs = plt.subplots(1, 3, figsize=(15,15))
axs[0].imshow(wzorA, cmap = 'gray')
axs[0].axis('off')
axs[0].set_title('wzor A')
axs[1].imshow(literki, cmap = 'gray')
axs[1].axis('off')
axs[1].set_title('literki')
axs[2].imshow(idft, cmap = 'gray')
axs[2].axis('off')
axs[2].set_title('idft')
fig.show()
    
